In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import EfficientFrontier, risk_models, expected_returns, plotting
import cvxpy as cp
from pypfopt.exceptions import OptimizationError

# Load data
emissions_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\Emissions Data_66 stocks.csv")
returns_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\Market Price_66 stocks.csv", index_col='Date', parse_dates=True, usecols=range(67))
sector_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\sector.csv")

# Data Cleansing
emissions_data.columns = emissions_data.columns.str.strip()
emissions_data = emissions_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
emissions_data['CO2 Equivalent Emissions Direct, Scope 1'] = emissions_data['CO2 Equivalent Emissions Direct, Scope 1'].str.replace(',', '').astype(float)
emissions_data['CO2 Equivalent Emissions Indirect, Scope 2'] = emissions_data['CO2 Equivalent Emissions Indirect, Scope 2'].str.replace(',', '').astype(float)

# Calculate total emissions for each instrument
emissions_data['Total Emissions'] = emissions_data['CO2 Equivalent Emissions Direct, Scope 1'] + emissions_data['CO2 Equivalent Emissions Indirect, Scope 2']
relevant_emissions = emissions_data.set_index('Instrument')['Total Emissions']

# Ensure the tickers in returns_data match those in relevant_emissions
tickers = returns_data.columns.intersection(relevant_emissions.index)

# Filter returns_data and relevant_emissions to include only matching tickers
returns_data_filtered = returns_data[tickers]
relevant_emissions_filtered = relevant_emissions[tickers]

# Calculate mean historical returns and sample covariance matrix
mu = expected_returns.mean_historical_return(returns_data_filtered, frequency=12, returns_data=True)
S = risk_models.risk_matrix(returns_data_filtered, method='ledoit_wolf', returns_data=True)

In [25]:
print(mu) #Mean Historical returns

AA     0.165041
AEO    0.072326
AIZ    0.136892
AKR   -0.053175
AMP    0.283265
         ...   
WHR   -0.001174
WOR    0.248323
WSR    0.063667
XHR   -0.048077
XOM    0.163286
Length: 66, dtype: float64


In [26]:
print(S) #Sample Covariance Matrix

           AA       AEO       AIZ       AKR       AMP       APH       APO  \
AA   8.649444  2.608937  0.820894  3.586521  2.427404  2.102477  2.015700   
AEO  2.608937  4.787959  0.622959  2.148176  1.448149  1.028997  1.524838   
AIZ  0.820894  0.622959  1.303265  0.439531  0.371896  0.305162  0.335225   
AKR  3.586521  2.148176  0.439531  3.522731  1.436860  1.083036  1.515414   
AMP  2.427404  1.448149  0.371896  1.436860  2.191635  1.072006  1.500877   
..        ...       ...       ...       ...       ...       ...       ...   
WHR  2.501548  1.498721  0.597294  1.212828  1.315306  1.250646  1.221500   
WOR  2.707792  1.910117  0.303637  1.494845  1.416197  1.144124  1.173015   
WSR  3.304596  1.406121  0.538150  2.183253  1.263198  0.977856  1.055865   
XHR  3.624285  2.086911  0.472853  2.951096  1.531916  1.099141  1.413029   
XOM  2.083039  0.925740  0.114781  1.445982  1.089304  0.685072  1.222861   

          AVB       AWR       AXP  ...       THS       TYL        UE  \
AA 

In [27]:
#Optimize for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))
weights = ef.max_sharpe()

In [28]:
# Convert weights to a DataFrame
tangency_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])

# Ensure the index is named 'Tickers'
tangency_weights.index.name = 'Tickers'

# Filter and sort the weights
filtered_weights = tangency_weights[tangency_weights['Weight'] > 0].sort_values(by='Weight', ascending=False)

# Reset the index to convert the index into a column
filtered_weights = filtered_weights.reset_index()

# Rename the columns
filtered_weights.columns = ['Tickers', 'Weight']

# Save the DataFrame to a CSV file
filtered_weights.to_csv('tangency_weights.csv', index=False)

In [29]:
#Calculate tangency portfolio returns
tangency_returns = (tangency_weights.T.values*returns_data).sum(axis=1)
print(tangency_returns)

Date
2019-06-01    0.085919
2019-07-01   -0.024340
2019-08-01    0.044941
2019-09-01   -0.006753
2019-10-01    0.033331
2019-11-01    0.024436
2019-12-01   -0.008887
2020-01-01    0.009473
2020-02-01   -0.063588
2020-03-01   -0.055932
2020-04-01    0.080577
2020-05-01    0.147482
2020-06-01    0.036470
2020-07-01    0.024030
2020-08-01    0.082817
2020-09-01   -0.032907
2020-10-01   -0.011245
2020-11-01    0.155068
2020-12-01    0.019044
2021-01-01    0.009201
2021-02-01    0.021131
2021-03-01    0.066145
2021-04-01    0.020547
2021-05-01    0.028610
2021-06-01    0.036616
2021-07-01    0.019762
2021-08-01    0.036571
2021-09-01   -0.023959
2021-10-01    0.064320
2021-11-01    0.002713
2021-12-01    0.029966
2022-01-01   -0.058578
2022-02-01    0.038413
2022-03-01    0.036750
2022-04-01   -0.078124
2022-05-01    0.011008
2022-06-01   -0.015492
2022-07-01    0.100709
2022-08-01    0.016655
2022-09-01   -0.075477
2022-10-01    0.108479
2022-11-01    0.024443
2022-12-01   -0.056180
2023-0

In [30]:
print(tangency_weights)

           Weight
Tickers          
AA       0.000000
AEO      0.000000
AIZ      0.041829
AKR      0.000000
AMP      0.000000
...           ...
WHR      0.000000
WOR      0.000000
WSR      0.000000
XHR      0.000000
XOM      0.000000

[66 rows x 1 columns]


In [31]:
#Calculating performance metrics 

mu_tangency_monthly = tangency_returns.mean()
mu_tangency_annual = (1+mu_tangency_monthly)**12-1

sigma_sq_tangency = tangency_returns.var(ddof=1)
sigma_tangency_monthly = tangency_returns.std(ddof=1)
sigma_tangency_annual = sigma_tangency_monthly*np.sqrt(12)

sr_tangency = mu_tangency_monthly/sigma_tangency_monthly*np.sqrt(12)

num_inv_tangency = np.sum(tangency_weights.values>0)
num_sectors_tangency = sector_data.merge(tangency_weights.loc[tangency_weights['Weight']>0], 
                                        left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

In [32]:
print(f"mu_tangency_annual: {mu_tangency_annual}")
print(f"sigma_tangency_annual: {sigma_tangency_annual}")
print(f"sr_tangency: {sr_tangency}")
print(f"num_inv_tangency: {num_inv_tangency}")
print(f"num_sectors_tangency: {num_sectors_tangency}")

mu_tangency_annual: 0.33065086313143044
sigma_tangency_annual: 0.17372802365015497
sr_tangency: 1.6640700425700523
num_inv_tangency: 11
num_sectors_tangency: 6


In [33]:
tangency_sector_weights = tangency_weights.merge(sector_data, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()
print("Tangency Sector Weights:\n", tangency_sector_weights)

Tangency Sector Weights:
 TRBC Economic Sector Name
Basic Materials           0.000000
Consumer Cyclicals        0.282539
Consumer Non-Cyclicals    0.117865
Energy                    0.024224
Financials                0.095747
Healthcare                0.000000
Industrials               0.139074
Real Estate               0.000000
Technology                0.340552
Utilities                 0.000000
Name: Weight, dtype: float64


In [34]:
#### Raw Emissions Tangency based Portfolio 

In [35]:
import pandas as pd

# Load the data
emissions_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\Emissions Data_66 stocks.csv")
returns_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\Market Price_66 stocks.csv", index_col='Date', parse_dates=True, usecols=range(67))
weights_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\tgncy_weights.csv")  # Load the weights from the generated CSV

# Strip leading and trailing spaces from the column names of emissions_data
emissions_data.columns = emissions_data.columns.str.strip()

# Strip leading and trailing spaces from the 'Instrument' column values
emissions_data['Instrument'] = emissions_data['Instrument'].str.strip()

# Set the index for easy access to rows based on Instrument
emissions_data.set_index('Instrument', inplace=True)

# Ensure the correct column names in weights_data
weights_data.columns = ['Tickers', 'Weight']

# Convert weights to a Series for easy manipulation
weights = pd.Series(weights_data.set_index('Tickers')['Weight'])

# Ensure matching data types for the indices
returns_data.columns = returns_data.columns.astype(str)
emissions_data.index = emissions_data.index.astype(str)
weights.index = weights.index.astype(str)

# Filter emissions data to match the tickers in returns_data
direct_emissions = emissions_data.loc[returns_data.columns, 'CO2 Equivalent Emissions Direct, Scope 1']
indirect_emissions = emissions_data.loc[returns_data.columns, 'CO2 Equivalent Emissions Indirect, Scope 2']

# Remove commas and convert emissions data to float to ensure correct data types
direct_emissions = direct_emissions.str.replace(',', '').astype(float)
indirect_emissions = indirect_emissions.str.replace(',', '').astype(float)

# Calculate the total emissions for each ticker
total_emissions = direct_emissions + indirect_emissions

# Ensure weights are float
weights = weights.astype(float)

# Calculate the weighted emissions for the portfolio
# Reindex weights to match total_emissions index
weights_reindexed = weights.reindex(total_emissions.index).fillna(0)

# Ensure weights_reindexed and total_emissions are float
weights_reindexed = weights_reindexed.astype(float)
total_emissions = total_emissions.astype(float)

# Perform the multiplication
weighted_emissions = weights_reindexed * total_emissions

# Calculate the sum of weighted emissions to get the 'Raw Emission of Market Cap Based Portfolio'
raw_emission_tangency_based_portfolio = weighted_emissions.sum()

# Create a DataFrame to store the results and save to CSV
emission_results = pd.DataFrame({
    'Ticker': total_emissions.index,
    'Weight': weights_reindexed,
    'Direct Emissions': direct_emissions,
    'Indirect Emissions': indirect_emissions,
    'Total Emissions': total_emissions,
    'Weighted Emissions': weighted_emissions
})

emission_results.to_csv('raw_emission_tangency_based_portfolio.csv', index=False)

In [36]:
# Assuming 'weights' contains the tangency portfolio weights
# Ensure weights are correctly aligned with returns_data columns
tangency_weights = pd.Series(weights, index=returns_data.columns).reindex(returns_data.columns, fill_value=0).values

# Calculate the tangency portfolio returns
tangency_returns = (returns_data * tangency_weights).sum(axis=1)

# Calculate the monthly and annual mean returns for the tangency portfolio
mu_tangency_monthly = tangency_returns.mean()
mu_tangency_annual = (1 + mu_tangency_monthly) ** 12 - 1

# Calculate the monthly and annual volatility (standard deviation) for the tangency portfolio
sigma_tangency_monthly = tangency_returns.std(ddof=1)
sigma_tangency_annual = sigma_tangency_monthly * np.sqrt(12)

# Calculate the Sharpe Ratio for the tangency portfolio (assuming risk-free rate is 0)
sr_tangency = mu_tangency_monthly / sigma_tangency_monthly * np.sqrt(12)

# Count the number of investments and sectors in the tangency portfolio
num_inv_tangency = np.sum(tangency_weights > 0)
num_sectors_tangency = len(np.unique(sector_data.loc[sector_data['Instrument'].isin(returns_data.columns[tangency_weights > 0]), 'TRBC Economic Sector Name']))

# Print the top 15 weights for the tangency portfolio
top15_weights = pd.Series(tangency_weights, index=returns_data.columns).sort_values(ascending=False).head(15)
print("Top 15 Weights:\n", top15_weights)
print(f"Monthly Mean Return: {mu_tangency_monthly:.4f}")
print(f"Annual Mean Return: {mu_tangency_annual:.4f}")
print(f"Monthly Volatility: {sigma_tangency_monthly:.4f}")
print(f"Annual Volatility: {sigma_tangency_annual:.4f}")
print(f"Sharpe Ratio: {sr_tangency:.4f}")
print(f"Number of Investments: {num_inv_tangency}")
print(f"Number of Sectors: {num_sectors_tangency}")


Top 15 Weights:
 BJ     0.282539
BAH    0.167469
HWM    0.139074
NOW    0.088939
GE     0.071816
TYL    0.064191
APO    0.053919
HRB    0.046049
AIZ    0.041829
HES    0.024224
APH    0.019953
AA          NaN
AEO         NaN
AKR         NaN
AMP         NaN
dtype: float64
Monthly Mean Return: 0.0241
Annual Mean Return: 0.3307
Monthly Volatility: 0.0502
Annual Volatility: 0.1737
Sharpe Ratio: 1.6641
Number of Investments: 11
Number of Sectors: 6


In [37]:
# Calculate the benchmark returns (e.g., an equally weighted portfolio or a specific benchmark)
benchmark_weights = np.ones(len(returns_data.columns)) / len(returns_data.columns)
benchmark_returns = (returns_data * benchmark_weights).sum(axis=1)

# Calculate the monthly and annual mean returns for the benchmark
mu_benchmark_monthly = benchmark_returns.mean()
mu_benchmark_annual = (1 + mu_benchmark_monthly) ** 12 - 1

# Calculate the monthly and annual volatility (standard deviation) for the benchmark
sigma_benchmark_monthly = benchmark_returns.std(ddof=1)
sigma_benchmark_annual = sigma_benchmark_monthly * np.sqrt(12)

# Calculate the Sharpe Ratio for the benchmark (assuming risk-free rate is 0)
sr_benchmark = mu_benchmark_monthly / sigma_benchmark_monthly * np.sqrt(12)
                                                                        
# Define the benchmark emissions (e.g., an equally weighted portfolio or a specific benchmark)
benchmark_emissions = (total_emissions * benchmark_weights).sum()

# Count the number of investments and sectors in the benchmark
num_inv_benchmark = np.sum(benchmark_weights > 0)
num_sectors_benchmark = returns_data.columns.nunique()

# Print the top 15 weights (though for an equally weighted portfolio, all weights are the same)                                                                  
print("Top 15 Weights:\n", benchmark_weights)
print(f"Monthly Mean Return: {mu_benchmark_monthly}")
print(f"Annual Mean Return: {mu_benchmark_annual}")
print(f"Monthly Volatility: {sigma_benchmark_monthly}")
print(f"Annual Volatility: {sigma_benchmark_annual}")
print(f"Sharpe Ratio: {sr_benchmark}")
print(f"Number of Investments: {num_inv_benchmark}")
print(f"Number of Sectors: {num_sectors_benchmark}")

Top 15 Weights:
 [0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152
 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152 0.01515152]
Monthly Mean Return: 0.014277600506310523
Annual Mean Return: 0.1854466524986762
Monthly Volatility: 0.06908643322599198
Annual Volatility: 0.23932242492226544
Sharpe Ratio: 0.7159011786353766
Number of Investments: 66
Number of Sectors: 66


In [38]:
# Assuming benchmark_weights is a numpy array
benchmark_weights = np.ones(len(returns_data.columns)) / len(returns_data.columns)

# Convert the numpy array to a pandas DataFrame
benchmark_weights_df = pd.DataFrame(benchmark_weights, index=returns_data.columns, columns=['Weight'])

# Save the DataFrame to a CSV file
benchmark_weights_df.to_csv('benchmark_weights.csv', index=True)

print("Benchmark weights have been saved as 'benchmark_weights.csv'.")

Benchmark weights have been saved as 'benchmark_weights.csv'.


In [39]:
#### Constrained Portfolio - 10% Reduced Emissions 

In [45]:
# Ensure Carbon Intensity values are strings, remove commas, and convert to float
emissions_data['Carbon Intensity'] = emissions_data['Carbon Intensity'].astype(str).str.replace(',', '').astype(float)

# Define constraints
def TE(w, portfolio_returns, benchmark_returns):
    xi = (w @ portfolio_returns.T - benchmark_returns.values.reshape(-1))
    mean = cp.sum(xi) / len(benchmark_returns)
    return cp.sum_squares(xi - mean)

def CI(w, portfolio_intensity):
    xi = w @ portfolio_intensity.T
    carbon_intensity = cp.sum(xi)
    return carbon_intensity

ef_constrained1 = EfficientFrontier(mu, S, weight_bounds=(0, 1))

ef_constrained1.add_constraint(lambda w: TE(w, returns_data, benchmark_returns) <= 0.1**2)
ef_constrained1.add_constraint(lambda w: CI(w, emissions_data['Carbon Intensity']) <= 0.9 * benchmark_emissions)
weights = ef_constrained1.convex_objective(TE, portfolio_returns=returns_data, benchmark_returns=benchmark_returns)

#Calculate the total emissionsfor the constrained portfolio
constrained1_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])
constrained1_weights[constrained1_weights['Weight']>0].sort_values(by='Weight', ascending=False)

constrained1_returns = (constrained1_weights.T.values*returns_data).sum(axis=1)

mu_constrained1_monthly = constrained1_returns.mean()
mu_constrained1_annual = (1+mu_constrained1_monthly)**12-1

sigma_sq_constrained1 = constrained1_returns.var(ddof=1)
sigma_constrained1_monthly = constrained1_returns.std(ddof=1)
sigma_constrained1_annual = sigma_constrained1_monthly*np.sqrt(12)

sr_constrained1 = mu_constrained1_monthly/sigma_constrained1_monthly*np.sqrt(12)

constrained1_te = (constrained1_returns - benchmark_returns.T).values.std(ddof=1)

num_inv_constrained1 = np.sum(constrained1_weights.values>10**-4)
num_sectors_constrained1 = sector_data.merge(constrained1_weights.loc[constrained1_weights['Weight']>10**-4], 
                                        left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

#Calculate the emissions for the constrained portfolio
constrained1_emissions = (constrained1_weights['Weight'].values * total_emissions.values).sum()

c1_sector_weights = constrained1_weights.merge(sector_data, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()

print("Top 15 Weights:\n", constrained1_weights)
print(f"Monthly Mean Return: {mu_constrained1_monthly}")
print(f"Annual Mean Return: {mu_constrained1_annual}")
print(f"Monthly Volatility: {sigma_constrained1_monthly}")
print(f"Annual Volatility: {sigma_constrained1_annual}")
print(f"Sharpe Ratio: {sr_constrained1}")
print(f"Tracking Error: {constrained1_te}")
print(f"Number of Investments: {num_inv_constrained1}")
print(f"Number of Sectors: {num_sectors_constrained1}")
print(f"Total Emissions: {constrained1_emissions}")
print("Sector Weights:\n", c1_sector_weights)

Top 15 Weights:
        Weight
AA   0.015176
AEO  0.015170
AIZ  0.015347
AKR  0.014853
AMP  0.014562
..        ...
WHR  0.014699
WOR  0.015443
WSR  0.014959
XHR  0.015738
XOM  0.014381

[66 rows x 1 columns]
Monthly Mean Return: 0.014267903227892073
Annual Mean Return: 0.18531065420670734
Monthly Volatility: 0.06908643322850791
Annual Volatility: 0.23932242493098088
Sharpe Ratio: 0.7154149419306702
Tracking Error: 2.0558436444026207e-09
Number of Investments: 66
Number of Sectors: 10
Total Emissions: 4579818.028216136
Sector Weights:
 TRBC Economic Sector Name
Basic Materials           0.061133
Consumer Cyclicals        0.166789
Consumer Non-Cyclicals    0.106870
Energy                    0.060278
Financials                0.122121
Healthcare                0.030122
Industrials               0.075038
Real Estate               0.150960
Technology                0.106028
Utilities                 0.120659
Name: Weight, dtype: float64


In [61]:
# Ensure that the DataFrame has the correct structure if necessary
if isinstance(constrained1_weights, pd.DataFrame):
    constrained1_weights.columns = ['Weight']
else:
    constrained1_weights = pd.DataFrame(constrained1_weights, columns=['Weight'])

# Save the DataFrame to a CSV file
constrained1_weights.to_csv('constrained1_weights.csv', index=True)

print("Constrained1 weights have been saved as 'constrained1_weights.csv'.")

Constrained1 weights have been saved as 'constrained1_weights.csv'.


In [105]:
#### Constrained Portfolio - 20% Carbon Intensity Reduction 

In [47]:
ef_constrained2 = EfficientFrontier(mu, S, weight_bounds=(0, 1))

ef_constrained2.add_constraint(lambda w: TE(w, returns_data, benchmark_returns) <= 0.1**2)
ef_constrained2.add_constraint(lambda w: CI(w, emissions_data['Carbon Intensity']) <= 0.8*benchmark_emissions)
weights = ef_constrained2.convex_objective(TE, portfolio_returns= returns_data, benchmark_returns=benchmark_returns)
constrained2_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])
constrained2_weights[constrained2_weights['Weight']>10**-6].sort_values(by='Weight', ascending=False)

constrained2_returns = (constrained2_weights.T.values*returns_data).sum(axis=1)

mu_constrained2_monthly = constrained2_returns.mean()
mu_constrained2_annual = (1+mu_constrained2_monthly)**12-1

sigma_sq_constrained2 =constrained2_returns.var(ddof=1)
sigma_constrained2_monthly = constrained2_returns.std(ddof=1)
sigma_constrained2_annual = sigma_constrained2_monthly*np.sqrt(12)

sr_constrained2 = mu_constrained2_monthly/sigma_constrained2_monthly*np.sqrt(12)

constrained2_te = (constrained2_returns - benchmark_returns.T).values.std(ddof=1)

num_inv_constrained2 = np.sum(constrained2_weights.values>10**-6)
num_sectors_constrained2 = sector_data.merge(constrained2_weights.loc[constrained2_weights['Weight']>10**-6], 
                                        left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()
constrained2_emissions = (constrained2_weights['Weight'].values * total_emissions.values).sum()

c2_sector_weights = constrained2_weights.merge(sector_data, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()

print("Top 15 Weights:\n", constrained2_weights)
print(f"Monthly Mean Return: {mu_constrained2_monthly}")
print(f"Annual Mean Return: {mu_constrained2_annual}")
print(f"Monthly Volatility: {sigma_constrained2_monthly}")
print(f"Annual Volatility: {sigma_constrained2_annual}")
print(f"Sharpe Ratio: {sr_constrained2}")
print(f"Tracking Error: {constrained2_te}")
print(f"Number of Investments: {num_inv_constrained2}")
print(f"Number of Sectors: {num_sectors_constrained2}")
print(f"Total Emissions: {constrained2_emissions}")
print("Sector Weights:\n", c2_sector_weights)

Top 15 Weights:
        Weight
AA   0.015264
AEO  0.015153
AIZ  0.015284
AKR  0.014847
AMP  0.014611
..        ...
WHR  0.014861
WOR  0.015313
WSR  0.014995
XHR  0.015568
XOM  0.014813

[66 rows x 1 columns]
Monthly Mean Return: 0.014276926788361408
Annual Mean Return: 0.18543720356158633
Monthly Volatility: 0.06908643329853811
Annual Volatility: 0.2393224251735726
Sharpe Ratio: 0.7158673966139274
Tracking Error: 2.636899256598795e-09
Number of Investments: 66
Number of Sectors: 10
Total Emissions: 4634849.923964959
Sector Weights:
 TRBC Economic Sector Name
Basic Materials           0.061032
Consumer Cyclicals        0.166540
Consumer Non-Cyclicals    0.106552
Energy                    0.060515
Financials                0.121580
Healthcare                0.030125
Industrials               0.075172
Real Estate               0.151218
Technology                0.106442
Utilities                 0.120824
Name: Weight, dtype: float64


In [60]:
# Ensure that the DataFrame has the correct structure if necessary
if isinstance(constrained2_weights, pd.DataFrame):
    constrained2_weights.columns = ['Weight']
else:
    constrained2_weights = pd.DataFrame(constrained2_weights, columns=['Weight'])

# Save the DataFrame to a CSV file
constrained2_weights.to_csv('constrained2_weights.csv', index=True)

print("Constrained2 weights have been saved as 'constrained2_weights.csv'.")

Constrained2 weights have been saved as 'constrained2_weights.csv'.


In [113]:
#### Constrained Portfolio - 50% Reduction in Carbon Intensity

In [50]:
ef_constrained3 = EfficientFrontier(mu, S, weight_bounds=(0, 1))

ef_constrained3.add_constraint(lambda w: TE(w, returns_data, benchmark_returns) <= 0.1**2)
ef_constrained3.add_constraint(lambda w: CI(w, emissions_data['Carbon Intensity']) <= 0.5*benchmark_emissions)
weights = ef_constrained3.convex_objective(TE, portfolio_returns=returns_data, benchmark_returns=benchmark_returns)

constrained3_weights = pd.DataFrame.from_dict(weights, orient='index', columns=['Weight'])
constrained3_weights[constrained3_weights['Weight']>10**-4].sort_values(by='Weight', ascending=False).head(15)
# Ensure all relevant columns are numeric
emissions_data = emissions_data.apply(lambda x: x.str.replace(',', '').astype(float) if x.dtype == 'object' else x)

# Calculate the returns for the constrained portfolio
constrained3_returns = (constrained3_weights['Weight'].values * returns_data).sum(axis=1)

# Calculate the monthly and annual mean returns
mu_constrained3_monthly = constrained3_returns.mean()
mu_constrained3_annual = (1 + mu_constrained3_monthly) ** 12 - 1

# Calculate the monthly and annual standard deviation (volatility)
sigma_sq_constrained3 = constrained3_returns.var(ddof=1)
sigma_constrained3_monthly = constrained3_returns.std(ddof=1)
sigma_constrained3_annual = sigma_constrained3_monthly * np.sqrt(12)

# Calculate the Sharpe Ratio
sr_constrained3 = mu_constrained3_monthly / sigma_constrained3_monthly * np.sqrt(12)

# Calculate the tracking error
constrained3_te = (constrained3_returns - benchmark_returns.values).std(ddof=1)

# Count the number of investments and sectors in the constrained portfolio
num_inv_constrained3 = np.sum(constrained3_weights['Weight'].values > 10**-4)
num_sectors_constrained3 = sector_data.merge(constrained3_weights[constrained3_weights['Weight'] > 10**-4], 
                                           left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

# Calculate the total emissions for the constrained portfolio
constrained3_emissions = (constrained3_weights['Weight'].values * emissions_data.sum(axis=1).values).sum()

c3_sector_weights = constrained3_weights.merge(sector_data, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()

print("Top 15 Weights:\n", constrained3_weights)
print(f"Monthly Mean Return: {mu_constrained3_monthly}")
print(f"Annual Mean Return: {mu_constrained3_annual}")
print(f"Monthly Volatility: {sigma_constrained3_monthly}")
print(f"Annual Volatility: {sigma_constrained3_annual}")
print(f"Sharpe Ratio: {sr_constrained3}")
print(f"Tracking Error: {constrained3_te}")
print(f"Number of Investments: {num_inv_constrained3}")
print(f"Number of Sectors: {num_sectors_constrained3}")
print(f"Total Emissions: {constrained3_emissions}")
print("Sector Weights:\n", c3_sector_weights)

Top 15 Weights:
        Weight
AA   0.015272
AEO  0.015131
AIZ  0.015361
AKR  0.014748
AMP  0.014553
..        ...
WHR  0.014811
WOR  0.015370
WSR  0.014978
XHR  0.015630
XOM  0.014692

[66 rows x 1 columns]
Monthly Mean Return: 0.014275879525680648
Annual Mean Return: 0.18542251577294588
Monthly Volatility: 0.06908643325633862
Annual Volatility: 0.2393224250273893
Sharpe Ratio: 0.7158148856654871
Tracking Error: 1.5726654094290995e-09
Number of Investments: 66
Number of Sectors: 10
Total Emissions: 27694690838.613724
Sector Weights:
 TRBC Economic Sector Name
Basic Materials           0.061038
Consumer Cyclicals        0.166649
Consumer Non-Cyclicals    0.106681
Energy                    0.060428
Financials                0.121746
Healthcare                0.030129
Industrials               0.075022
Real Estate               0.151178
Technology                0.106348
Utilities                 0.120781
Name: Weight, dtype: float64


In [59]:
# Ensure that the DataFrame has the correct structure if necessary
if isinstance(constrained3_weights, pd.DataFrame):
    constrained3_weights.columns = ['Weight']
else:
    constrained3_weights = pd.DataFrame(constrained3_weights, columns=['Weight'])

# Save the DataFrame to a CSV file
constrained3_weights.to_csv('constrained3_weights.csv', index=True)

print("Constrained3 weights have been saved as 'constrained3_weights.csv'.")

Constrained3 weights have been saved as 'constrained3_weights.csv'.


In [53]:
#### Contrained Portfolio - Tangency Portfolio with 20% less Carbon Intensity

In [54]:
import pandas as pd
import numpy as np
from pypfopt import expected_returns, risk_models

# Load data
emissions_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\Emissions Data_66 stocks.csv")
returns_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\Market Price_66 stocks.csv", index_col='Date', parse_dates=True, usecols=range(67))
sector_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\sector.csv")

# Data Cleansing
emissions_data.columns = emissions_data.columns.str.strip()
emissions_data = emissions_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
emissions_data['CO2 Equivalent Emissions Direct, Scope 1'] = emissions_data['CO2 Equivalent Emissions Direct, Scope 1'].str.replace(',', '').astype(float)
emissions_data['CO2 Equivalent Emissions Indirect, Scope 2'] = emissions_data['CO2 Equivalent Emissions Indirect, Scope 2'].str.replace(',', '').astype(float)

# Calculate total emissions for each instrument
emissions_data['Total Emissions'] = emissions_data['CO2 Equivalent Emissions Direct, Scope 1'] + emissions_data['CO2 Equivalent Emissions Indirect, Scope 2']

# Convert Carbon Intensity to numeric
emissions_data['Carbon Intensity'] = emissions_data['Carbon Intensity'].str.replace(',', '').astype(float)

# Ensure the tickers in returns_data match those in emissions_data
tickers = returns_data.columns.intersection(emissions_data['Instrument'])

# Filter returns_data and emissions_data to include only matching tickers
returns_data_filtered = returns_data[tickers]
emissions_data_filtered = emissions_data.set_index('Instrument').loc[tickers]

# Calculate mean historical returns and sample covariance matrix
mu = expected_returns.mean_historical_return(returns_data_filtered, frequency=12, returns_data=True)
S = risk_models.risk_matrix(returns_data_filtered, method='ledoit_wolf', returns_data=True)

# Define constraints
def calculate_portfolio_return(weights, mu):
    return np.dot(weights, mu)

def calculate_portfolio_volatility(weights, S):
    return np.sqrt(np.dot(weights.T, np.dot(S, weights)))

def calculate_sharpe_ratio(weights, mu, S, risk_free_rate=0.02):
    portfolio_return = calculate_portfolio_return(weights, mu)
    portfolio_volatility = calculate_portfolio_volatility(weights, S)
    return (portfolio_return - risk_free_rate) / portfolio_volatility

def calculate_carbon_intensity(weights, carbon_intensity):
    return np.dot(weights, carbon_intensity)

# Heuristic optimization
n_assets = len(mu)
best_sharpe_ratio = -np.inf
best_weights = np.zeros(n_assets)
carbon_intensity_threshold = 0.8 * emissions_data_filtered['Carbon Intensity'].mean()

for _ in range(10000):  # Number of iterations
    weights = np.random.random(n_assets)
    weights /= np.sum(weights)  # Normalize weights to sum to 1
    
    portfolio_sharpe_ratio = calculate_sharpe_ratio(weights, mu, S)
    portfolio_carbon_intensity = calculate_carbon_intensity(weights, emissions_data_filtered['Carbon Intensity'].values)
    
    if portfolio_carbon_intensity <= carbon_intensity_threshold and portfolio_sharpe_ratio > best_sharpe_ratio:
        best_sharpe_ratio = portfolio_sharpe_ratio
        best_weights = weights

# Convert weights to a DataFrame
constrained4_weights = pd.DataFrame(best_weights, index=returns_data_filtered.columns, columns=['Weight'])
top15_constrained4_weights = constrained4_weights[constrained4_weights['Weight'] > 10**-4].sort_values(by='Weight', ascending=False).head(15)
print("Top 15 Weights:\n", top15_constrained4_weights)

# Calculate returns for the constrained portfolio
constrained4_returns = (constrained4_weights.T.values * returns_data_filtered).sum(axis=1)

# Calculate monthly and annual mean returns
mu_constrained4_monthly = constrained4_returns.mean()
mu_constrained4_annual = (1 + mu_constrained4_monthly) ** 12 - 1

# Calculate monthly and annual standard deviation (volatility)
sigma_sq_constrained4 = constrained4_returns.var(ddof=1)
sigma_constrained4_monthly = constrained4_returns.std(ddof=1)
sigma_constrained4_annual = sigma_constrained4_monthly * np.sqrt(12)

# Calculate Sharpe Ratio
sr_constrained4 = mu_constrained4_monthly / sigma_constrained4_monthly * np.sqrt(12)

# Calculate tracking error
benchmark_returns = returns_data_filtered.mean(axis=1)  # Assuming benchmark returns are the average returns
constrained4_te = (constrained4_returns - benchmark_returns.values).std(ddof=1)

# Count the number of investments and sectors in the constrained portfolio
num_inv_constrained4 = np.sum(constrained4_weights['Weight'].values > 10**-4)
num_sectors_constrained4 = sector_data.merge(constrained4_weights[constrained4_weights['Weight'] > 10**-4], 
                                             left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

# Calculate the total emissions for the constrained portfolio
constrained4_emissions = (constrained4_weights['Weight'].values * emissions_data_filtered['Total Emissions'].values).sum()

# Calculate sector weights
c4_sector_weights = constrained4_weights.merge(sector_data, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()

# Print the results
print(f"mu_constrained4_annual: {mu_constrained4_annual}")
print(f"sigma_constrained4_annual: {sigma_constrained4_annual}")
print(f"sr_constrained4: {sr_constrained4}")
print(f"constrained4_te: {constrained4_te}")
print(f"num_inv_constrained4: {num_inv_constrained4}")
print(f"num_sectors_constrained4: {num_sectors_constrained4}")
print(f"constrained4_emissions: {constrained4_emissions}")
print("Sector Weights:\n", c4_sector_weights)


Top 15 Weights:
        Weight
APO  0.034530
FND  0.033296
CRM  0.033029
IR   0.031701
HWM  0.031326
NFG  0.030992
WOR  0.028388
BJ   0.027178
IQV  0.026887
TDY  0.026873
CNC  0.026564
BV   0.026284
MRO  0.024875
FDX  0.024395
GE   0.024077
mu_constrained4_annual: 0.217914977917802
sigma_constrained4_annual: 0.23523412269174737
sr_constrained4: 0.8449820585469756
constrained4_te: 0.007454248075683635
num_inv_constrained4: 66
num_sectors_constrained4: 10
constrained4_emissions: 2817901.081397816
Sector Weights:
 TRBC Economic Sector Name
Basic Materials           0.086019
Consumer Cyclicals        0.149716
Consumer Non-Cyclicals    0.106715
Energy                    0.054715
Financials                0.093398
Healthcare                0.053451
Industrials               0.125246
Real Estate               0.081890
Technology                0.134851
Utilities                 0.113999
Name: Weight, dtype: float64


In [62]:
# Ensure that the DataFrame has the correct structure if necessary
if isinstance(constrained4_weights, pd.DataFrame):
    constrained4_weights.columns = ['Weight']
else:
    constrained4_weights = pd.DataFrame(constrained4_weights, columns=['Weight'])

# Save the DataFrame to a CSV file
constrained4_weights.to_csv('constrained4_weights.csv', index=True)

print("Constrained4 weights have been saved as 'constrained4_weights.csv'.")

Constrained4 weights have been saved as 'constrained4_weights.csv'.


In [56]:
#### Constrained Portfolio - Tangency w/ 20% CI reduction & Sector Balance

In [80]:
import pandas as pd
import numpy as np
from pypfopt import expected_returns, risk_models

# Assuming 'emissions_data', 'sector_data', 'returns_data', 'mu', 'S', and 'tangency_returns' are preloaded
# Ensure 'emissions_data' and 'sector_data' have numeric columns where expected
emissions_data = emissions_data.apply(pd.to_numeric, errors='coerce')
sector_data['Instrument'] = sector_data['Instrument'].astype(str)
returns_data.columns = returns_data.columns.astype(str)

# Assuming benchmark_weights is a numpy array
benchmark_weights_df = pd.DataFrame(benchmark_weights, index=returns_data.columns, columns=['Weight'])

# Merge with sector data
sector_weights = benchmark_weights_df.merge(sector_data, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()

lower_bound = (sector_weights * 0.7).to_dict()
upper_bound = (sector_weights * 1.3).to_dict()

# Create the sector_mapper dictionary
sector_mapper = sector_data.set_index('Instrument')['TRBC Economic Sector Name'].to_dict()

# Manually adjust weights
weights = np.ones(len(returns_data.columns)) / len(returns_data.columns)
constrained_weights = pd.DataFrame(weights, index=returns_data.columns, columns=['Weight'])

# Define constraints
total_emissions = (emissions_data['CO2 Equivalent Emissions Direct, Scope 1'] + emissions_data['CO2 Equivalent Emissions Indirect, Scope 2']).sum()
emissions_constraint = 0.8 * total_emissions

# Adjust weights to meet carbon intensity constraint
while True:
    weighted_emissions = (constrained_weights['Weight'].values * emissions_data['CO2 Equivalent Emissions Direct, Scope 1'].values).sum() + \
                         (constrained_weights['Weight'].values * emissions_data['CO2 Equivalent Emissions Indirect, Scope 2'].values).sum()
    if weighted_emissions <= emissions_constraint:
        break
    # Adjust weights by reducing the highest carbon intensity weights
    max_emission_index = (constrained_weights['Weight'].values * (emissions_data['CO2 Equivalent Emissions Direct, Scope 1'] + emissions_data['CO2 Equivalent Emissions Indirect, Scope 2']).values).argmax()
    constrained_weights.iloc[max_emission_index] *= 0.95
    constrained_weights /= constrained_weights.sum()  # Normalize to sum to 1

# Ensure sector constraints are met
for sector in sector_weights.index:
    sector_indices = [i for i, x in enumerate(constrained_weights.index) if sector_mapper[x] == sector]
    sector_sum = constrained_weights.iloc[sector_indices].sum().values[0]
    if sector_sum < lower_bound[sector]:
        constrained_weights.iloc[sector_indices] *= (lower_bound[sector] / sector_sum)
    elif sector_sum > upper_bound[sector]:
        constrained_weights.iloc[sector_indices] *= (upper_bound[sector] / sector_sum)
    constrained_weights /= constrained_weights.sum()  # Normalize to sum to 1

# Calculate returns for the manually adjusted portfolio
constrained5_returns = (constrained_weights.T.values * returns_data.values).sum(axis=1)

mu_constrained5_monthly = constrained5_returns.mean()
mu_constrained5_annual = (1 + mu_constrained5_monthly) ** 12 - 1

sigma_sq_constrained5 = constrained5_returns.var(ddof=1)
sigma_constrained5_monthly = constrained5_returns.std(ddof=1)
sigma_constrained5_annual = sigma_constrained5_monthly * np.sqrt(12)

sr_constrained5 = mu_constrained5_monthly / sigma_constrained5_monthly * np.sqrt(12)

constrained5_te = (constrained5_returns - tangency_returns).std(ddof=1)

num_inv_constrained5 = np.sum(constrained_weights.values > 0)
num_sectors_constrained5 = sector_data.merge(constrained_weights[constrained_weights['Weight'] > 0], 
                                             left_on='Instrument', right_index=True)['TRBC Economic Sector Name'].nunique()

constrained5_emissions = (constrained_weights['Weight'].values * (emissions_data['CO2 Equivalent Emissions Direct, Scope 1'] + emissions_data['CO2 Equivalent Emissions Indirect, Scope 2']).values).sum()
c5_sector_weights = constrained_weights.merge(sector_data, left_index=True, right_on='Instrument').groupby('TRBC Economic Sector Name')['Weight'].sum()

print(f"Constrained Portfolio Weights:\n{constrained_weights}")
print(f"Monthly Mean Return: {mu_constrained5_monthly}")
print(f"Annual Mean Return: {mu_constrained5_annual}")
print(f"Monthly Volatility: {sigma_constrained5_monthly}")
print(f"Annual Volatility: {sigma_constrained5_annual}")
print(f"Sharpe Ratio: {sr_constrained5}")
print(f"Tracking Error: {constrained5_te}")
print(f"Number of Investments: {num_inv_constrained5}")
print(f"Number of Sectors: {num_sectors_constrained5}")
print(f"Total Emissions: {constrained5_emissions}")
print(f"Sector Weights:\n{c5_sector_weights}")


Constrained Portfolio Weights:
       Weight
AA   0.015152
AEO  0.015152
AIZ  0.015152
AKR  0.015152
AMP  0.015152
..        ...
WHR  0.015152
WOR  0.015152
WSR  0.015152
XHR  0.015152
XOM  0.015152

[66 rows x 1 columns]
Monthly Mean Return: 0.014277600506310523
Annual Mean Return: 0.1854466524986762
Monthly Volatility: 0.06908643322599198
Annual Volatility: 0.23932242492226544
Sharpe Ratio: 0.7159011786353766
Tracking Error: 0.041950616598922706
Number of Investments: 66
Number of Sectors: 10
Total Emissions: 4686645.745
Sector Weights:
TRBC Economic Sector Name
Basic Materials           0.060606
Consumer Cyclicals        0.166667
Consumer Non-Cyclicals    0.106061
Energy                    0.060606
Financials                0.121212
Healthcare                0.030303
Industrials               0.075758
Real Estate               0.151515
Technology                0.106061
Utilities                 0.121212
Name: Weight, dtype: float64


In [82]:
# Save the constrained weights to CSV
constrained_weights.to_csv('constrained5_weights.csv', index=True)

print("Constrained5 weights have been saved as 'constrained5_weights.csv'.")

Constrained5 weights have been saved as 'constrained5_weights.csv'.


In [139]:
#### Summary Dataframe Composition

In [225]:
import pandas as pd

# Create individual DataFrames for each portfolio with the provided data
bp = pd.DataFrame({
    'Market Cap-Weighted': [mu_benchmark_monthly, mu_benchmark_annual, sigma_benchmark_monthly,
                            sigma_benchmark_annual, sr_benchmark, num_inv_benchmark, num_sectors_benchmark, '', 0]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

tp = pd.DataFrame({
    'Tangency': [mu_tangency_monthly, mu_tangency_annual, sigma_tangency_monthly,
                 sigma_tangency_annual, sr_tangency, num_inv_tangency, num_sectors_tangency, '', 0]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

p1 = pd.DataFrame({
    'Market Cap-Weighted w/ 10% CI Reduction': [mu_constrained1_monthly, mu_constrained1_annual, sigma_constrained1_monthly,
                                                sigma_constrained1_annual, sr_constrained1, num_inv_constrained1, num_sectors_constrained1, 'Market-Cap', 100*constrained1_te]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

p2 = pd.DataFrame({
    'Market Cap-Weighted w/ 20% CI Reduction': [mu_constrained2_monthly, mu_constrained2_annual, sigma_constrained2_monthly,
                                                sigma_constrained2_annual, sr_constrained2, num_inv_constrained2, num_sectors_constrained2, 'Market-Cap', 100*constrained2_te]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

p3 = pd.DataFrame({
    'Market Cap-Weighted w/ 50% CI Reduction': [mu_constrained3_monthly, mu_constrained3_annual, sigma_constrained3_monthly,
                                                sigma_constrained3_annual, sr_constrained3, num_inv_constrained3, num_sectors_constrained3, 'Market-Cap', 100*constrained3_te]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

p4 = pd.DataFrame({
    'Tangency w/ 20% CI Reduction': [mu_constrained4_monthly, mu_constrained4_annual, sigma_constrained4_monthly,
                                     sigma_constrained4_annual, sr_constrained4, num_inv_constrained4, num_sectors_constrained4, 'Tangency', 100*constrained4_te]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

p5 = pd.DataFrame({
    'Tangency w/ 20% CI Reduction & Sector Balance': [mu_constrained5_monthly, mu_constrained5_annual, sigma_constrained5_monthly,
                                                      sigma_constrained5_annual, sr_constrained5, num_inv_constrained5, num_sectors_constrained5, 'Tangency', 100*constrained5_te]
}, index=['Monthly Return', 'Annual Return', 'Monthly Volatility', 'Annual Volatility',
          'Sharpe Ratio', 'Number of Investments', 'Number of Sectors', 'Benchmark', 'Tracking Error']).T

# Concatenate all DataFrames
df_summary = pd.concat([bp, tp, p1, p2, p3, p4, p5])

# Format the DataFrame
df_summary = df_summary.style.format({
    'Monthly Return': '{:,.4f}',
    'Annual Return': '{:,.4f}',
    'Monthly Volatility': '{:,.4f}',
    'Annual Volatility': '{:,.4f}',
    'Sharpe Ratio': '{:,.4f}',
    'Number of Investments': '{:,.0f}',
    'Number of Sectors': '{:,.0f}',
    'Tracking Error': '{:,.4f}%'
})

# Convert styled DataFrame back to regular DataFrame
df_summary_regular = df_summary.data

# Save the DataFrame to a CSV file
df_summary_regular.to_csv('df_summary.csv', index=True)

# Display the styled DataFrame
df_summary


,Monthly Return,Annual Return,Monthly Volatility,Annual Volatility,Sharpe Ratio,Number of Investments,Number of Sectors,Benchmark,Tracking Error
Market Cap-Weighted,0.0143,0.1854,0.0691,0.2393,0.7159,66,66,,0.0000%
Tangency,0.0241,0.3307,0.0502,0.1737,1.6641,11,6,,0.0000%
Market Cap-Weighted w/ 10% CI Reduction,0.0139,0.1797,0.0691,0.2393,0.6954,66,3,Market-Cap,0.0000%
Market Cap-Weighted w/ 20% CI Reduction,0.0137,0.1773,0.0691,0.2393,0.6866,66,3,Market-Cap,0.0000%
Market Cap-Weighted w/ 50% CI Reduction,0.0137,0.1770,0.0691,0.2393,0.6856,58,10,Market-Cap,0.0186%
Tangency w/ 20% CI Reduction,0.0165,0.2169,0.0670,0.2321,0.8529,65,10,Tangency,0.6771%
Tangency w/ 20% CI Reduction & Sector Balance,0.0218,0.2948,0.0511,0.1770,1.4757,14,10,Tangency,1.1281%


In [ ]:
#ESG Score based on the benchmark weights 

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import EfficientFrontier, risk_models, expected_returns, plotting
import cvxpy as cp
from pypfopt.exceptions import OptimizationError

#Import data 
esg_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\ESG.csv")
sector_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\sector.csv")
benchmark_weights = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\benchmark_weights.csv")


#Data Cleansing
esg_data.columns = esg_data.columns.str.strip()
sector_data.columns = sector_data.columns.str.strip()
benchmark_weights.columns = benchmark_weights.columns.str.strip()

#Merge Dataframes 
esg_data.set_index('Instrument', inplace=True)
sector_data.set_index('Instrument', inplace=True)
benchmark_weights.set_index('Unnamed: 0', inplace=True)

# Merge dataframes
df_esg_sector_bm = esg_data.merge(sector_data, left_index=True, right_index=True).merge(benchmark_weights, left_index=True, right_index=True)

# Calculate weighted scores
df_esg_sector_bm['ESG Score'] = df_esg_sector_bm['Weight'] * df_esg_sector_bm['ESG Score']
df_esg_sector_bm['Social Pillar Score'] = df_esg_sector_bm['Weight'] * df_esg_sector_bm['Social Pillar Score']
df_esg_sector_bm['Environmental Pillar Score'] = df_esg_sector_bm['Weight'] * df_esg_sector_bm['Environmental Pillar Score']
df_esg_sector_bm['Governance Pillar Score'] = df_esg_sector_bm['Weight'] * df_esg_sector_bm['Governance Pillar Score']

print(df_esg_sector_bm.head())

     ESG Score  Environmental Pillar Score  Social Pillar Score  \
AA    1.324067                    1.332292             1.232163   
AXP   1.049161                    0.525584             1.141203   
VZ    1.075870                    1.130561             1.109311   
XOM   1.052247                    0.986701             1.073818   
GE    1.183011                    1.193895             1.082999   

     Governance Pillar Score TRBC Economic Sector Name    Weight  
AA                  1.462899           Basic Materials  0.015152  
AXP                 1.131779                Financials  0.015152  
VZ                  0.966874                Technology  0.015152  
XOM                 1.109911                    Energy  0.015152  
GE                  1.329847    Consumer Non-Cyclicals  0.015152  


In [84]:
# ESG score based on Constraints 

In [89]:
import pandas as pd
import numpy as np

# Load the datasets
esg_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\ESG.csv")
sector_data = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\sector.csv")
benchmark_weights = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\benchmark_weights.csv")
constrained1_weights = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\constrained1_weights.csv")
constrained2_weights = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\constrained2_weights.csv")
constrained3_weights = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\constrained3_weights.csv")
constrained4_weights = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\constrained4_weights.csv")
constrained5_weights = pd.read_csv(r"C:\Users\adhit\OneDrive\desktop\GDS\Green Data Science project\DATA\constrained5_weights.csv")

# Check the columns of each DataFrame
print("ESG Data Columns:", esg_data.columns)
print("Sector Data Columns:", sector_data.columns)
print("Benchmark Weights Columns:", benchmark_weights.columns)
print("Constrained1 Weights Columns:", constrained1_weights.columns)
print("Constrained2 Weights Columns:", constrained2_weights.columns)
print("Constrained3 Weights Columns:", constrained3_weights.columns)
print("Constrained4 Weights Columns:", constrained4_weights.columns)
print("Constrained5 Weights Columns:", constrained5_weights.columns)

# Rename 'Unnamed: 0' to 'Instrument' in the weights DataFrames
benchmark_weights.rename(columns={'Unnamed: 0': 'Instrument'}, inplace=True)
constrained1_weights.rename(columns={'Unnamed: 0': 'Instrument'}, inplace=True)
constrained2_weights.rename(columns={'Unnamed: 0': 'Instrument'}, inplace=True)
constrained3_weights.rename(columns={'Unnamed: 0': 'Instrument'}, inplace=True)
constrained4_weights.rename(columns={'Unnamed: 0': 'Instrument'}, inplace=True)
constrained5_weights.rename(columns={'Unnamed: 0': 'Instrument'}, inplace=True)

# Ensure the weights DataFrames have 'Instrument' as the index
benchmark_weights.set_index('Instrument', inplace=True)
constrained1_weights.set_index('Instrument', inplace=True)
constrained2_weights.set_index('Instrument', inplace=True)
constrained3_weights.set_index('Instrument', inplace=True)
constrained4_weights.set_index('Instrument', inplace=True)
constrained5_weights.set_index('Instrument', inplace=True)

# Merge ESG and sector data
df_esg_sector = esg_data.merge(sector_data, on='Instrument')

# Function to calculate weighted ESG scores
def calculate_weighted_scores(df, weights):
    df_merged = df.merge(weights, left_on='Instrument', right_index=True)
    df_merged['Weighted ESG Score'] = df_merged['Weight'] * df_merged['ESG Score']
    df_merged['Weighted Social Pillar Score'] = df_merged['Weight'] * df_merged['Social Pillar Score']
    df_merged['Weighted Environmental Pillar Score'] = df_merged['Weight'] * df_merged['Environmental Pillar Score']
    df_merged['Weighted Governance Pillar Score'] = df_merged['Weight'] * df_merged['Governance Pillar Score']
    df_merged.drop(columns=['Weight', 'Instrument'], inplace=True)
    return df_merged

# Calculate weighted ESG scores for each set of weights
df_esg_sector_bm = calculate_weighted_scores(df_esg_sector, benchmark_weights)
df_esg_sector_cs1 = calculate_weighted_scores(df_esg_sector, constrained1_weights)
df_esg_sector_cs2 = calculate_weighted_scores(df_esg_sector, constrained2_weights)
df_esg_sector_cs3 = calculate_weighted_scores(df_esg_sector, constrained3_weights)
df_esg_sector_cs4 = calculate_weighted_scores(df_esg_sector, constrained4_weights)
df_esg_sector_cs5 = calculate_weighted_scores(df_esg_sector, constrained5_weights)

# Group by sector and sum the scores
esg_grouped_bm = df_esg_sector_bm.groupby('TRBC Economic Sector Name').sum()
esg_grouped_cs1 = df_esg_sector_cs1.groupby('TRBC Economic Sector Name').sum()
esg_grouped_cs2 = df_esg_sector_cs2.groupby('TRBC Economic Sector Name').sum()
esg_grouped_cs3 = df_esg_sector_cs3.groupby('TRBC Economic Sector Name').sum()
esg_grouped_cs4 = df_esg_sector_cs4.groupby('TRBC Economic Sector Name').sum()
esg_grouped_cs5 = df_esg_sector_cs5.groupby('TRBC Economic Sector Name').sum()

# Normalize the grouped data to sum to 100
esg_grouped_bm = esg_grouped_bm / esg_grouped_bm.sum() * 100
esg_grouped_cs1 = esg_grouped_cs1 / esg_grouped_cs1.sum() * 100
esg_grouped_cs2 = esg_grouped_cs2 / esg_grouped_cs2.sum() * 100
esg_grouped_cs3 = esg_grouped_cs3 / esg_grouped_cs3.sum() * 100
esg_grouped_cs4 = esg_grouped_cs4 / esg_grouped_cs4.sum() * 100
esg_grouped_cs5 = esg_grouped_cs5 / esg_grouped_cs5.sum() * 100

# Merge all grouped dataframes
esg_grouped = esg_grouped_bm.merge(esg_grouped_cs1, left_index=True, right_index=True, suffixes=[' - Market', ' - Constrained1'])
esg_grouped = esg_grouped.merge(esg_grouped_cs2, left_index=True, right_index=True, suffixes=['', ' - Constrained2'])
esg_grouped = esg_grouped.merge(esg_grouped_cs3, left_index=True, right_index=True, suffixes=['', ' - Constrained3'])
esg_grouped = esg_grouped.merge(esg_grouped_cs4, left_index=True, right_index=True, suffixes=['', ' - Constrained4'])
esg_grouped = esg_grouped.merge(esg_grouped_cs5, left_index=True, right_index=True, suffixes=['', ' - Constrained5'])

# Reorder columns as specified
cols = esg_grouped.columns
new_cols = [
    cols[0], cols[4], cols[8], cols[12], cols[16], cols[20],  # ESG Scores
    cols[1], cols[5], cols[9], cols[13], cols[17], cols[21],  # Social Pillar Scores
    cols[2], cols[6], cols[10], cols[14], cols[18], cols[22], # Environmental Pillar Scores
    cols[3], cols[7], cols[11], cols[15], cols[19], cols[23]  # Governance Pillar Scores
]
esg_grouped = esg_grouped[new_cols]

# Display the resulting DataFrame
print(esg_grouped)

ESG Data Columns: Index(['Instrument', 'ESG Score', 'Environmental Pillar Score',
       'Social Pillar Score', 'Governance Pillar Score'],
      dtype='object')
Sector Data Columns: Index(['Instrument', 'TRBC Economic Sector Name'], dtype='object')
Benchmark Weights Columns: Index(['Unnamed: 0', 'Weight'], dtype='object')
Constrained1 Weights Columns: Index(['Unnamed: 0', 'Weight'], dtype='object')
Constrained2 Weights Columns: Index(['Unnamed: 0', 'Weight'], dtype='object')
Constrained3 Weights Columns: Index(['Unnamed: 0', 'Weight'], dtype='object')
Constrained4 Weights Columns: Index(['Unnamed: 0', 'Weight'], dtype='object')
Constrained5 Weights Columns: Index(['Unnamed: 0', 'Weight'], dtype='object')
                           ESG Score - Market  Weighted ESG Score - Market  \
TRBC Economic Sector Name                                                    
Basic Materials                      5.956569                     5.956569   
Consumer Cyclicals                  14.936813      

In [93]:
# Save the resulting DataFrame to a CSV file

esg_grouped.to_csv('esg_grouped.csv', index=True)
print(f"The grouped ESG data comparison has been saved as 'esg_grouped'")

The grouped ESG data comparison has been saved as 'esg_grouped'
